# Lab Assignment 5

- Name - Aryan Gupta
- Roll No.- 230150003
- Date - 3 Sept., 2025
- Course - DA331 Big Data Analytics: Tools & Techniques

## Importing Libraries

In [5]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, sum, hour, countDistinct

spark = SparkSession.builder.appName("lab5").getOrCreate()

# read all the filenames available
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


## Load Data

In [6]:
# Get where the session is built
!pwd

/kaggle/working


In [9]:
# define the paths of the file
path1 = '/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv'
path2 = '/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv'

In [ ]:
# load data
df1 = spark.read.csv(path1, header=True, inferSchema=True)
df2 = spark.read.csv(path2, header=True, inferSchema=True)

In [12]:
# print basic things
print('DF1')
print(df1.printSchema())
print(df1.count())
print(len(df1.columns))
print(df1.show(5))

print('============================')

print('DF2')
print(df2.printSchema())
print(df2.count())
print(len(df2.columns))
print(df2.show(5))

DF1
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

None


67501979
9
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.smar

42448764
9
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|205301355563188

## Converting 'event_type' column to numeric

In [15]:
# checking all the event types
event_types = df1.select("event_type").distinct().rdd.flatMap(lambda x: x).collect()
print(event_types)

['purchase', 'view', 'cart']


- 'view' = 0
- 'cart' = 1
- 'purchase' = 2

In [18]:
# converting to numerical column
df1 = df1.withColumn("event_type_numeric", when(col("event_type") == "view", 0)
                                         .when(col("event_type") == "cart", 1)
                                         .when(col("event_type") == "purchase", 2)
                                         .otherwise(None))

df2 = df2.withColumn("event_type_numeric", when(col("event_type") == "view", 0)
                                         .when(col("event_type") == "cart", 1)
                                         .when(col("event_type") == "purchase", 2)
                                         .otherwise(None))

In [24]:
df1.show(5)
df2.show(5)

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|event_type_numeric|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|                 0|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|                 0|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|                 0|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|51808559

## Q1 - Are the top 10 by view-to-cart-conversion rate category-of-product the same for both the months?

In [25]:
# Calculate view and cart counts for df1
category_conversion_nov = df1.groupBy("category_code").agg(
    count(when(col("event_type_numeric") == 0, True)).alias("view_count_nov"),
    count(when(col("event_type_numeric") == 1, True)).alias("cart_count_nov")
)

# Calculate conversion rate for df1
category_conversion_nov = category_conversion_nov.withColumn(
    "conversion_rate_nov",
    (col("cart_count_nov") / col("view_count_nov"))
)

print("Category Conversion Rates (November):")
category_conversion_nov.show(5)

# Calculate view and cart counts for df2
category_conversion_oct = df2.groupBy("category_code").agg(
    count(when(col("event_type_numeric") == 0, True)).alias("view_count_oct"),
    count(when(col("event_type_numeric") == 1, True)).alias("cart_count_oct")
)

# Calculate conversion rate for df2
category_conversion_oct = category_conversion_oct.withColumn(
    "conversion_rate_oct",
    (col("cart_count_oct") / col("view_count_oct"))
)

print("Category Conversion Rates (October):")
category_conversion_oct.show()

Category Conversion Rates (November):


+--------------------+--------------+--------------+--------------------+
|       category_code|view_count_nov|cart_count_nov| conversion_rate_nov|
+--------------------+--------------+--------------+--------------------+
|    computers.ebooks|         45761|          1870|0.040864491597648656|
|apparel.shoes.sli...|          5552|            52|0.009365994236311239|
|computers.periphe...|         84909|          2709| 0.03190474508002685|
|electronics.video...|         22423|           418|  0.0186415733844713|
|appliances.kitche...|         18206|           559| 0.03070416346259475|
+--------------------+--------------+--------------+--------------------+
only showing top 5 rows

Category Conversion Rates (October):


+--------------------+--------------+--------------+--------------------+
|       category_code|view_count_oct|cart_count_oct| conversion_rate_oct|
+--------------------+--------------+--------------+--------------------+
|apparel.shoes.sli...|          5677|             0|                 0.0|
|    computers.ebooks|         29083|           476|0.016366949764467214|
|computers.periphe...|         52565|           421|0.008009131551412536|
|electronics.video...|         12925|           147| 0.01137330754352031|
|appliances.kitche...|         11585|           222|0.019162710401381098|
|     sport.snowboard|          3747|             0|                 0.0|
|electronics.camer...|         39172|           510|0.013019503727152047|
|       apparel.shirt|         13476|             0|                 0.0|
|electronics.audio...|       1018542|         51143|0.050211969658590414|
|appliances.kitche...|         82114|           975|0.011873736512653141|
|appliances.kitche...|         46240| 

In [27]:
# get top 10 categories
q1_categories_nov = category_conversion_nov.select(['category_code', 'conversion_rate_nov'])\
            .sort(col('conversion_rate_nov').desc()).limit(10)\
            .select('category_code').rdd.flatMap(lambda x: x).collect()

q1_categories_oct = category_conversion_oct.select(['category_code', 'conversion_rate_oct'])\
            .sort(col('conversion_rate_oct').desc()).limit(10)\
            .select('category_code').rdd.flatMap(lambda x: x).collect()

In [32]:
# printList function for ease of access
def printList(x):
  for i in x[:len(x)-1]:
    print(i, end=', ')
  print(x[len(x)-1])

In [34]:
print('Top 10 categories in November:')
print(f'length={len(q1_categories_nov)}')
printList(q1_categories_nov)
print('================================')
print('Top 10 categories in October:')
print(f'length={len(q1_categories_oct)}')
printList(q1_categories_oct)
print('================================')

# seperating common categories from categories appearing only once
inNov = []
inOct = []
inBoth = []

for cat in q1_categories_nov:
    # if in both months
    if cat in q1_categories_oct:
        inBoth.append(cat)
    else:
        inNov.append(cat)
for cat in q1_categories_oct:
    # if in both months, already added once
    if cat in inBoth:
        continue
    else:
        inOct.append(cat)

print('Categories in November but not in October:')
print(f'length={len(inNov)}')
printList(inNov)
print('================================')
print('Categories in October but not in November:')
print(f'length={len(inOct)}')
printList(inOct)
print('================================')
print('Categories in both October and November:')
print(f'length={len(inBoth)}')
printList(inBoth)
print('================================')
print(f'Total Categories: {len(inNov)+len(inOct)+len(inBoth)}')
print(f'Therefore, only {len(inBoth)} out of {len(inNov)+len(inOct)+len(inBoth)} categories are common in both months.')

Top 10 categories in November:
length=10
kids.fmcg.diapers, appliances.kitchen.fryer, electronics.audio.headphone, electronics.smartphone, appliances.environment.water_heater, medicine.tools.tonometer, appliances.kitchen.microwave, appliances.personal.scales, stationery.cartrige, appliances.kitchen.washer
Top 10 categories in October:
length=10
electronics.smartphone, electronics.audio.headphone, medicine.tools.tonometer, electronics.video.tv, electronics.tablet, appliances.kitchen.microwave, appliances.environment.water_heater, appliances.kitchen.washer, appliances.iron, computers.peripherals.printer
Categories in November but not in October:
length=4
kids.fmcg.diapers, appliances.kitchen.fryer, appliances.personal.scales, stationery.cartrige
Categories in October but not in November:
length=4
electronics.video.tv, electronics.tablet, appliances.iron, computers.peripherals.printer
Categories in both October and November:
length=6
electronics.audio.headphone, electronics.smartphone, ap

## Q2 - Top 10 category-of-product that have increasing revenue and decreasing revenue for the two consecutive months

I am assuming that the question means most increasing and most decreasing revenue, and not to compare increasing or decreasing revenue from the categories in question 1.

In [35]:
# Calculate revenue by category for df1 (November)
category_revenue_nov = df1.filter(col("event_type_numeric") == 2).groupBy("category_code").agg(
    sum("price").alias("total_revenue_nov")
)

print("Category Revenue (November):")
category_revenue_nov.show()

# Calculate revenue by category for df2 (October)
category_revenue_oct = df2.filter(col("event_type_numeric") == 2).groupBy("category_code").agg(
    sum("price").alias("total_revenue_oct")
)

print("Category Revenue (October):")
category_revenue_oct.show()

Category Revenue (November):


+--------------------+------------------+
|       category_code| total_revenue_nov|
+--------------------+------------------+
|    computers.ebooks|64230.219999999994|
|computers.periphe...|28976.320000000007|
|appliances.kitche...| 4721.330000000001|
|electronics.video...|          55364.94|
|electronics.camer...|         157889.89|
|       apparel.shirt| 6849.440000000002|
|electronics.audio...| 5669502.490000021|
|appliances.kitche...| 415451.8099999999|
|appliances.kitche...| 79856.06999999999|
|appliances.kitche...|313824.69000000006|
|  electronics.tablet|1520253.0100000005|
|auto.accessories....| 44147.61000000001|
|apparel.shoes.moc...|           2225.06|
|       apparel.jeans|           6195.61|
|computers.periphe...|366142.17999999993|
|furniture.living_...|         649805.37|
| stationery.cartrige| 3212.479999999999|
|furniture.kitchen...|129180.86000000007|
|construction.tool...|278838.74000000005|
|furniture.bedroom...| 4931.980000000002|
+--------------------+------------

+--------------------+------------------+
|       category_code| total_revenue_oct|
+--------------------+------------------+
|apparel.shoes.sli...|398.20000000000005|
|    computers.ebooks|47831.619999999995|
|computers.periphe...|26891.610000000004|
|appliances.kitche...|           3683.97|
|electronics.video...| 37059.95000000001|
|electronics.camer...|140711.99000000002|
|electronics.audio...|3539126.9900000007|
|appliances.kitche...|255830.39000000007|
|appliances.kitche...|53324.380000000005|
|  electronics.tablet|1610973.7799999998|
|auto.accessories....| 40968.96000000001|
|appliances.kitche...|205012.05000000002|
|computers.periphe...|         228858.29|
|furniture.living_...|          452398.4|
| stationery.cartrige|2027.3800000000003|
|furniture.kitchen...|          79904.25|
|construction.tool...|230588.81000000006|
|furniture.bedroom...|3882.7399999999993|
|electronics.audio...|34709.759999999995|
|auto.accessories....| 83753.49999999999|
+--------------------+------------

In [36]:
# Join the revenue dataframes and calculate the revenue difference
category_revenue_comparison = category_revenue_oct.join(category_revenue_nov, "category_code", "outer").na.fill(0)
category_revenue_comparison = category_revenue_comparison.withColumn(
    "revenue_difference",
    col("total_revenue_nov") - col("total_revenue_oct")
)

### Increasing Revenue

In [39]:
increasing_revenue_categories = category_revenue_comparison.orderBy(col("revenue_difference").desc()).limit(10)

print("Top 10 Categories with Increasing Revenue (October to November):")
increasing_revenue_categories.show()

Top 10 Categories with Increasing Revenue (October to November):


+--------------------+--------------------+--------------------+--------------------+
|       category_code|   total_revenue_oct|   total_revenue_nov|  revenue_difference|
+--------------------+--------------------+--------------------+--------------------+
|                NULL|                 0.0| 2.988050667999996E7| 2.988050667999996E7|
|electronics.smart...|1.5704962336999914E8|1.7782166160999876E8|2.0772038239999622E7|
|electronics.video.tv|   8423407.859999994|1.2457151160000006E7|   4033743.300000012|
|electronics.audio...|  3539126.9900000007|   5669502.490000021|    2130375.50000002|
|  electronics.clocks|   4818305.469999999|   6552737.249999998|  1734431.7799999993|
|  computers.notebook|          8979887.25|1.0678429710000006E7|  1698542.4600000065|
|appliances.kitche...|   4658646.590000002|          5801906.33|  1143259.7399999984|
|appliances.enviro...|  1716425.4099999995|   2762311.619999999|  1045886.2099999997|
|appliances.kitche...|          3830077.01|   4722657.

### Decreasing Revenue

In [40]:
decreasing_revenue_categories = category_revenue_comparison.orderBy(col("revenue_difference").asc()).limit(10)

print("Top 10 Categories with Decreasing Revenue (October to November):")
decreasing_revenue_categories.show()

Top 10 Categories with Decreasing Revenue (October to November):


+--------------------+--------------------+------------------+--------------------+
|       category_code|   total_revenue_oct| total_revenue_nov|  revenue_difference|
+--------------------+--------------------+------------------+--------------------+
|                NULL|2.2924937129999954E7|               0.0|-2.29249371299999...|
|  electronics.tablet|  1610973.7799999998|1520253.0100000005|  -90720.76999999932|
|         kids.skates|  202582.28999999995|         118786.27|  -83796.01999999995|
|       sport.bicycle|  203694.72999999995|158871.60999999993| -44823.120000000024|
|auto.accessories....|   523241.7999999999| 487207.6299999999| -36034.169999999984|
|  apparel.shoes.keds|            188218.5|169055.61000000002| -19162.889999999985|
|construction.tool...|           173856.12|166201.41999999998|  -7654.700000000012|
|construction.tool...|            34624.83|          28959.86|  -5664.970000000001|
|construction.tool...|  14339.579999999994|           9605.15|  -4734.429999

## Q3 - Are the top 10 by view-to-cart-conversion rate brands the same for both the two months?

In [41]:
# Calculate view and cart counts for df1 (November) by brand
brand_conversion_nov = df1.groupBy("brand").agg(
    count(when(col("event_type_numeric") == 0, True)).alias("view_count_nov"),
    count(when(col("event_type_numeric") == 1, True)).alias("cart_count_nov")
)

# Calculate conversion rate for df1 by brand
brand_conversion_nov = brand_conversion_nov.withColumn(
    "conversion_rate_nov",
    (col("cart_count_nov") / col("view_count_nov"))
)

print("Brand Conversion Rates (November):")
brand_conversion_nov.show()

# Calculate view and cart counts for df2 (October) by brand
brand_conversion_oct = df2.groupBy("brand").agg(
    count(when(col("event_type_numeric") == 0, True)).alias("view_count_oct"),
    count(when(col("event_type_numeric") == 1, True)).alias("cart_count_oct")
)

# Calculate conversion rate for df2 by brand
brand_conversion_oct = brand_conversion_oct.withColumn(
    "conversion_rate_oct",
    (col("cart_count_oct") / col("view_count_oct"))
)

print("Brand Conversion Rates (October):")
brand_conversion_oct.show()

Brand Conversion Rates (November):


+-------------+--------------+--------------+--------------------+
|        brand|view_count_nov|cart_count_nov| conversion_rate_nov|
+-------------+--------------+--------------+--------------------+
|     yokohama|        117384|          8680| 0.07394534178422954|
|      edifier|          3491|            53| 0.01518189630478373|
|        welss|          4892|            29|0.005928045789043336|
|      tuffoni|          3570|            39|0.010924369747899159|
|         tega|          2261|            48|0.021229544449358692|
|      serebro|          6802|           256|0.037635989414877974|
|        crest|           316|             9|0.028481012658227847|
|         tmnt|           569|             4|0.007029876977152899|
|       alutec|          2516|            60| 0.02384737678855326|
|        sonel|         11007|           370| 0.03361497229036068|
|     nocnezna|           245|             3|0.012244897959183673|
|     nutricia|          2197|           229| 0.10423304506144

+--------+--------------+--------------+--------------------+
|   brand|view_count_oct|cart_count_oct| conversion_rate_oct|
+--------+--------------+--------------+--------------------+
|yokohama|        110521|          1758|0.015906479311624037|
|   welss|          2867|            17|0.005929543076386466|
| tuffoni|          2106|             0|                 0.0|
|    tmnt|           208|             0|                 0.0|
| serebro|          1348|             0|                 0.0|
| edifier|          2435|             7|0.002874743326488706|
|   globo|            31|             0|                 0.0|
|    tega|          1468|             0|                 0.0|
|  vortex|            46|             0|                 0.0|
|   sonel|          7081|             0|                 0.0|
|nutricia|          1644|             0|                 0.0|
| bombbar|           380|             0|                 0.0|
|  alutec|          2645|             0|                 0.0|
|   goo.

In [42]:
# Identify top 10 brands by conversion rate for November
top_brands_nov_conversion = brand_conversion_nov.orderBy(col("conversion_rate_nov").desc()).limit(10)

print("Top 10 Brands by Conversion Rate (November):")
top_brands_nov_conversion.show()

# Identify top 10 brands by conversion rate for October
top_brands_oct_conversion = brand_conversion_oct.orderBy(col("conversion_rate_oct").desc()).limit(10)

print("Top 10 Brands by Conversion Rate (October):")
top_brands_oct_conversion.show()

# Compare the top 10 brands by conversion rate
nov_brands_conversion = set(top_brands_nov_conversion.select("brand").rdd.flatMap(lambda x: x).collect())
oct_brands_conversion = set(top_brands_oct_conversion.select("brand").rdd.flatMap(lambda x: x).collect())

print("\nAre the top 10 brands by conversion rate the same for both months?")
print(nov_brands_conversion == oct_brands_conversion)

print("\nBrands in November's top 10 conversion but not in October's:")
print(nov_brands_conversion - oct_brands_conversion)

print("\nBrands in October's top 10 conversion but not in November's:")
print(oct_brands_conversion - nov_brands_conversion)

Top 10 Brands by Conversion Rate (November):


+------------+--------------+--------------+-------------------+
|       brand|view_count_nov|cart_count_nov|conversion_rate_nov|
+------------+--------------+--------------+-------------------+
| pixiebelles|             5|             2|                0.4|
|    autolite|             5|             2|                0.4|
|       ivaru|           177|            51|  0.288135593220339|
|   bigklever|             4|             1|               0.25|
|       lamel|             9|             2| 0.2222222222222222|
|   superfine|            19|             4|0.21052631578947367|
|merejbaspasy|            91|            19| 0.2087912087912088|
|         htl|             5|             1|                0.2|
|        basf|           126|            25| 0.1984126984126984|
|    yoshioki|            54|            10|0.18518518518518517|
+------------+--------------+--------------+-------------------+

Top 10 Brands by Conversion Rate (October):


+--------------------+--------------+--------------+--------------------+
|               brand|view_count_oct|cart_count_oct| conversion_rate_oct|
+--------------------+--------------+--------------+--------------------+
|                  ea|         13025|          1855| 0.14241842610364683|
|             diamond|           594|            44| 0.07407407407407407|
|             samsung|       4806630|        303249| 0.06308973230725061|
|         homeelement|           101|             6|  0.0594059405940594|
|               steba|            85|             5|0.058823529411764705|
|              domyos|          2178|           128| 0.05876951331496786|
|               goo.n|           933|            53| 0.05680600214362272|
|sonyinteractiveen...|           985|            55| 0.05583756345177665|
|               apple|       3770597|        209084| 0.05545116595594809|
|       dreammachines|            81|             4| 0.04938271604938271|
+--------------------+--------------+-


Are the top 10 brands by conversion rate the same for both months?
False

Brands in November's top 10 conversion but not in October's:
{'autolite', 'pixiebelles', 'htl', 'superfine', 'merejbaspasy', 'yoshioki', 'bigklever', 'ivaru', 'basf', 'lamel'}

Brands in October's top 10 conversion but not in November's:
{'dreammachines', 'apple', 'steba', 'homeelement', 'goo.n', 'ea', 'samsung', 'domyos', 'sonyinteractiveentertainment', 'diamond'}


## Q4 - Top 10 brands that have increasing revenue and decreasing revenue for the two consecutive months

Similar assumption as Q2, i.e, I am assuming that the question means most increasing and most decreasing revenue, and not to compare increasing or decreasing revenue from the brands in question 3.

In [43]:
# Calculate revenue by brand for df1 (November)
brand_revenue_nov = df1.filter(col("event_type_numeric") == 2).groupBy("brand").agg(
    sum("price").alias("total_revenue_nov")
)

print("Brand Revenue (November):")
brand_revenue_nov.show()

# Calculate revenue by brand for df2 (October)
brand_revenue_oct = df2.filter(col("event_type_numeric") == 2).groupBy("brand").agg(
    sum("price").alias("total_revenue_oct")
)

print("Brand Revenue (October):")
brand_revenue_oct.show()

Brand Revenue (November):


+--------+------------------+
|   brand| total_revenue_nov|
+--------+------------------+
|yokohama|244691.75999999998|
|    tega|             83.24|
| edifier|           2049.43|
|   welss| 625.8199999999999|
|nutricia| 546.6299999999999|
|   goo.n|252.18999999999997|
|   sonel|13520.890000000005|
| ariston|241908.44000000006|
|decoroom| 4903.979999999999|
| versace| 81352.02000000002|
| camshel|177.60000000000002|
|  rieker|60525.060000000005|
|  wagner|           4478.12|
|  xp-pen|12458.819999999998|
| philips|         658690.01|
|  yamaha| 280557.9000000001|
|logitech|11083.499999999998|
|davidoff|             385.8|
|   pride| 43416.37999999998|
| olympus|           1910.38|
+--------+------------------+
only showing top 20 rows

Brand Revenue (October):


+--------+------------------+
|   brand| total_revenue_oct|
+--------+------------------+
|yokohama|237257.95000000007|
| tuffoni|1305.8600000000001|
|nutricia|            636.17|
|   sonel|10586.080000000002|
| versace|          51521.47|
| ariston|192233.21999999997|
|decoroom|3424.7299999999987|
|   stagg| 9185.499999999996|
| cartier|2946.7200000000003|
|bioderma| 832.2800000000001|
|  rieker|28754.880000000005|
| philips|514433.39999999997|
| m-audio|13611.159999999996|
| ansmann| 449.1900000000001|
|  yamaha|         142948.13|
|logitech|10822.499999999998|
|nitecore|           1210.71|
|   pride|          24674.28|
|takamine| 7563.990000000001|
|   stihl| 56085.95999999999|
+--------+------------------+
only showing top 20 rows



In [44]:
# Join the revenue dataframes and calculate the revenue difference
brand_revenue_comparison = brand_revenue_oct.join(brand_revenue_nov, "brand", "outer").na.fill(0)
brand_revenue_comparison = brand_revenue_comparison.withColumn(
    "revenue_difference",
    col("total_revenue_nov") - col("total_revenue_oct")
)

### Increasing Revenue

In [45]:
increasing_revenue_brands = brand_revenue_comparison.orderBy(col("revenue_difference").desc()).limit(10)

print("Top 10 Brands with Increasing Revenue (October to November):")
increasing_revenue_brands.show()

Top 10 Brands with Increasing Revenue (October to November):


+-------+-------------------+--------------------+--------------------+
|  brand|  total_revenue_oct|   total_revenue_nov|  revenue_difference|
+-------+-------------------+--------------------+--------------------+
|  apple|1.112092688200001E8|1.2751252488000059E8|1.6303256060000494E7|
|   NULL|                0.0|1.1025718790000005E7|1.1025718790000005E7|
|samsung|4.640753261000007E7| 5.486988086999983E7|   8462348.259999767|
| xiaomi|  9194033.289999995|1.1259865960000008E7|   2065832.670000013|
|     lg| 3387887.9600000004|          5239018.76|  1851130.7999999993|
|   sony| 2478196.6799999997|   3862886.300000004|  1384689.6200000043|
|   oppo| 2412959.7600000002|  3488540.7599999965|  1075580.9999999963|
| lenovo| 1752638.5299999993|   2698106.299999999|   945467.7699999996|
|   asus|          970019.39|  1665811.5900000003|   695792.2000000003|
|  artel| 1034152.4099999999|   1642215.040000001|    608062.630000001|
+-------+-------------------+--------------------+--------------

### Decreasing Revenue

In [46]:
decreasing_revenue_brands = brand_revenue_comparison.orderBy(col("revenue_difference").asc()).limit(10)

print("Top 10 Brands with Decreasing Revenue (October to November):")
decreasing_revenue_brands.show()

Top 10 Brands with Decreasing Revenue (October to November):


+--------+------------------+------------------+-------------------+
|   brand| total_revenue_oct| total_revenue_nov| revenue_difference|
+--------+------------------+------------------+-------------------+
|    NULL| 8540601.030000001|               0.0| -8540601.030000001|
|    acer|3576719.5200000014| 3347306.530000001|-229412.99000000022|
|    peda|219342.36999999997|48278.939999999995|-171063.42999999996|
|omabelle| 347730.8700000001|201897.13000000003|-145833.74000000008|
|  hubert|219706.36000000002| 98882.01000000005|-120824.34999999996|
|  huawei| 4883421.739999999|        4780682.35|-102739.38999999966|
|elenberg|259771.00000000012|160643.44000000006| -99127.56000000006|
|    skad|189851.01999999996|         115344.12| -74506.89999999997|
|   force|286056.18999999994|222173.14999999994| -63883.04000000001|
|    beko| 963940.5899999999| 904347.0499999997|-59593.540000000154|
+--------+------------------+------------------+-------------------+



## Q5 - Top mid-night (12 midnight to 10 am) online flea-market brands by revenue in Oct

In [47]:
# Analyze midnight flea market revenue for df2 (October)
midnight_oct = df2.filter(hour(col("event_time")).between(0, 10) & (col("event_type_numeric") == 2))

brand_midnight_revenue_oct = midnight_oct.groupBy("brand").agg(
    sum("price").alias("midnight_revenue_oct")
).sort(col("midnight_revenue_oct").desc())

# making a set of the best brands
midnight_best_brands_oct = set(brand_midnight_revenue_oct.limit(10).select("brand")\
                            .rdd.flatMap(lambda x: x).collect())

print("Brand Revenue during Midnight Flea Market (October):")
brand_midnight_revenue_oct.show()

print("Top 10 Brands with Midnight Flea Market Revenue (October):")
print(midnight_best_brands_oct)

Brand Revenue during Midnight Flea Market (October):


+--------+--------------------+
|   brand|midnight_revenue_oct|
+--------+--------------------+
|   apple|  5.88757908200002E7|
| samsung|2.6419483550000086E7|
|  xiaomi|          5028192.67|
|    NULL|          4584259.91|
|  huawei|   2609788.219999999|
|    acer|  1899131.4700000004|
|      lg|  1881891.5699999996|
| lucente|  1770841.0499999993|
|    oppo|  1348583.6300000001|
|    sony|          1287472.23|
|  lenovo|   939205.7800000001|
| indesit|           731279.39|
|   bosch|   682422.8099999999|
|      hp|   670535.8099999998|
|   artel|   617066.0300000003|
|    beko|           544379.31|
|    asus|  509808.68999999994|
|   haier|  470484.94999999995|
|dauscher|            352857.5|
|cordiant|   312652.1799999999|
+--------+--------------------+
only showing top 20 rows

Top 10 Brands with Midnight Flea Market Revenue (October):
{'acer', 'apple', 'xiaomi', 'huawei', 'lucente', 'oppo', None, 'samsung', 'lg', 'sony'}


## Q6 - Top mid-night (12 midnight to 10 am) online flea-market brands by revenue in Nov

In [48]:
# Analyze midnight flea market revenue for df1 (November)
midnight_nov = df1.filter(hour(col("event_time")).between(0, 10) & (col("event_type_numeric") == 2))

brand_midnight_revenue_nov = midnight_nov.groupBy("brand").agg(
    sum("price").alias("midnight_revenue_nov")
).sort(col("midnight_revenue_nov").desc())

# making a set of the best brands
midnight_best_brands_nov = set(brand_midnight_revenue_nov.limit(10).select("brand")\
                            .rdd.flatMap(lambda x: x).collect())

print("Brand Revenue during Midnight Flea Market (November):")
brand_midnight_revenue_nov.show()

print("Top 10 Brands with Midnight Flea Market Revenue (November):")
print(midnight_best_brands_nov)

Brand Revenue during Midnight Flea Market (November):


+--------+--------------------+
|   brand|midnight_revenue_nov|
+--------+--------------------+
|   apple|6.2912121670000166E7|
| samsung| 2.864050041999994E7|
|  xiaomi|   5564491.590000002|
|    NULL|   5263266.450000001|
|      lg|          2679971.44|
|  huawei|  2337180.7699999996|
|    sony|  1807858.9700000007|
|    oppo|  1774335.7099999988|
| lucente|  1773218.7699999998|
|    acer|  1650349.2600000002|
|  lenovo|  1282633.1599999997|
|   artel|   876579.1999999997|
|    asus|           853841.19|
|   bosch|   849909.6700000002|
| indesit|   665359.6400000001|
|      hp|           662829.09|
|   haier|            553649.2|
|cordiant|   475437.2600000003|
|    beko|           464420.51|
|dauscher|  444740.52999999997|
+--------+--------------------+
only showing top 20 rows

Top 10 Brands with Midnight Flea Market Revenue (November):
{'acer', 'apple', 'xiaomi', 'huawei', 'lucente', 'oppo', None, 'samsung', 'lg', 'sony'}


## Q7 -  Are the brands the same for Oct and Nov for Question 3 and Question 4?

In [49]:
# Brands from Q3 are stored in nov_brands_conversion and oct_brands_conversion

# Get the brands from Q4 for each month, for both increasing and decreasing revenue
increasing_revenue_brands_set = set(increasing_revenue_brands.select("brand").rdd.flatMap(lambda x: x).collect())
decreasing_revenue_brands_set = set(decreasing_revenue_brands.select("brand").rdd.flatMap(lambda x: x).collect())

# Combine increasing and decreasing revenue brands for comparison
revenue_brands_nov = increasing_revenue_brands_set.union(decreasing_revenue_brands_set)
revenue_brands_oct = increasing_revenue_brands_set.union(decreasing_revenue_brands_set)

print("\nAre the top 10 brands by conversion rate the same as brands with increasing/decreasing revenue for November?")
print(nov_brands_conversion == revenue_brands_nov)

print("\nBrands in November's top 10 conversion but not in November's increasing/decreasing revenue:")
print(nov_brands_conversion - revenue_brands_nov)

print("\nBrands in November's increasing/decreasing revenue but not in November's top 10 conversion:")
print(revenue_brands_nov - nov_brands_conversion)

print("\nAre the top 10 brands by conversion rate the same as brands with increasing/decreasing revenue for October?")
print(oct_brands_conversion == revenue_brands_oct)

print("\nBrands in October's top 10 conversion but not in October's increasing/decreasing revenue:")
print(oct_brands_conversion - revenue_brands_oct)

print("\nBrands in October's increasing/decreasing revenue but not in October's top 10 conversion:")
print(revenue_brands_oct - oct_brands_conversion)


Are the top 10 brands by conversion rate the same as brands with increasing/decreasing revenue for November?
False

Brands in November's top 10 conversion but not in November's increasing/decreasing revenue:
{'autolite', 'pixiebelles', 'htl', 'superfine', 'merejbaspasy', 'yoshioki', 'bigklever', 'ivaru', 'basf', 'lamel'}

Brands in November's increasing/decreasing revenue but not in November's top 10 conversion:
{'apple', 'huawei', None, 'omabelle', 'force', 'xiaomi', 'asus', 'oppo', 'beko', 'hubert', 'samsung', 'lg', 'elenberg', 'lenovo', 'acer', 'peda', 'artel', 'sony', 'skad'}

Are the top 10 brands by conversion rate the same as brands with increasing/decreasing revenue for October?
False

Brands in October's top 10 conversion but not in October's increasing/decreasing revenue:
{'dreammachines', 'steba', 'homeelement', 'goo.n', 'ea', 'domyos', 'sonyinteractiveentertainment', 'diamond'}

Brands in October's increasing/decreasing revenue but not in October's top 10 conversion:
{'bek